In [1]:
import gzip
from collections import defaultdict
import math
import scipy.optimize
from sklearn import svm
import numpy as np
import numpy
import string
import random
import pandas as pd
import string
from sklearn import linear_model
from tqdm import tqdm
from types import NoneType
import pickle 
import matplotlib.pyplot as plt

In [2]:
def parseData(fname):
    for l in gzip.open(fname):
        d = eval(l)
        yield d

In [ ]:
#dt_places = parseData("places.clean.json.gz")
#dt_users = parseData("users.clean.json.gz")
#dt_reviews = parseData("reviews.clean.json.gz")

In [ ]:
#places = pd.DataFrame(dt_places)
#users = pd.DataFrame(dt_users)
#reviews = pd.DataFrame(dt_reviews)

In [18]:
#users = open("users.pkl",'rb')
#users = pickle.load(users)
#places = open("places.pkl",'rb')
#places = pickle.load(places)
#reviews = open("reviews.pkl",'rb')
#reviews = pickle.load(reviews)

In [3]:
dataset_dir = "../Datasets/"
merged = pickle.load(open(dataset_dir+"merged.pkl",'rb'))

In [4]:
print(len(merged))

8649011


In [5]:
merged.head()

,userName,gPlusUserId,rating,reviewText,categories,gPlusPlaceId,unixReviewTime,name,price,gps,closed
0,an lam,100000010817154263736,3.0,Chất lượng tạm ổn,[Giải Trí - Café],108103314380004200232,1.372687e+09,Cà Phê Thăng Long,None,"[10.852044, 106.65971]",False
1,hoang long nguyen,101659842775092396018,5.0,"Good coffee, nice and peaceful place",[Giải Trí - Café],108103314380004200232,1.354888e+09,Cà Phê Thăng Long,None,"[10.852044, 106.65971]",False
2,Hong Le,107574994242995460712,2.0,Cho heo,[Giải Trí - Café],108103314380004200232,1.352015e+09,Cà Phê Thăng Long,None,"[10.852044, 106.65971]",False
3,HALİL TURGUT,100000013500285534661,5.0,Wc si temiz duzenli..,[Turkish Cuisine],102194128241608748649,1.342871e+09,Selale Restaurant,None,"[37.8037, 29.2209]",False
4,Akudosoft Yazılım,105271324704942360981,5.0,None,[Turkish Cuisine],102194128241608748649,1.373148e+09,Selale Restaurant,None,"[37.8037, 29.2209]",False


In [6]:
temp_data = merged[0:1000].copy()

In [54]:
minnX = 10e9
maxxX = -10e9
minnY = 10e9
maxxY = -10e9
cnt_outliers  = 0 
for i in tqdm(range(len(merged))):
    if merged['gps'][i] is not None:
        if merged['gps'][i][0] <= -180 or merged['gps'][i][0] >= 360:
            cnt_outliers+=1
        minnX = min(minnX,merged['gps'][i][0])
        maxxX = max(maxxX,merged['gps'][i][0])
        minnY = min(minnY,merged['gps'][i][1])
        maxxY = max(maxxY,merged['gps'][i][1])
    
print(minnX,maxxX,minnY,maxxY)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8649011/8649011 [02:39<00:00, 54374.68it/s]

-54806068.0 69775847.0 -159601736.0 178054936.0



###

#values making no sense at all 
-54806068.0 69775847.0 -159601736.0 178054936.0

In [15]:
temp_data = merged.sample(frac=0.1)

In [17]:
temp_data = temp_data.reset_index(drop=True)

In [18]:
temp_data

,userName,gPlusUserId,rating,reviewText,categories,gPlusPlaceId,unixReviewTime,name,price,gps,closed
0,Justin Holland,104835831581429401434,4.0,Great job about a half hour wait for the $20.0...,[Car Wash],111799194080300135670,1.362161e+09,Civic Center Car Wash Inc,None,"[34.507753, -117.327177]",False
1,ronald de leon,110331607471448368479,5.0,excellent sushi and great service.,"[Asian Restaurant, Japanese Restaurant, Produc...",116252055894950252701,1.359046e+09,Haru Sushi,$$$,"[40.819809, -74.000241]",True
2,Michael Cairns,106185504400627678384,5.0,Have been using the team at Abercorn Plumbing ...,"[Plumber, Plumbing Supply Store]",105794257818107244229,1.349604e+09,Abercorn Plumbing & Heating,None,"[55.95843, -3.137685]",False
3,Rose Brown,113712395831134054373,2.0,None,[Post Office],115302093695316020601,1.369516e+09,Clementsport Po,None,"[44.665374, -65.610563]",False
4,전태현,108831174692729957795,5.0,괞찬음,[시외버스터미널],113786330221572304722,1.393477e+09,안산 종합여객자동차 터미널,None,"[37.316856, 126.846288]",False
...,...,...,...,...,...,...,...,...,...,...,...
864896,Bill Chen,107246420456791561435,5.0,None,"[Chinese Restaurant, Asian Restaurant]",113658954753439258377,1.375559e+09,Taiwan Cafe Inc,None,"[33.019414, -96.747739]",False
864897,Frank Spychalski,105929453737619095652,5.0,"Super-friendly host, very nice house, perfect ...","[Budget Hotel, Hostel]",112448682523588624165,1.352096e+09,Anakiwa Backpackers & Green Caravan Cafe,None,"[-41.263977, 173.920679]",False
864898,Phil Friesen,103571354677022020945,5.0,This store has an amazing selection of Asian f...,"[Asian Grocery Store, Supermarket]",118302020708442752531,1.374108e+09,Lotte Plaza - Ashburn,None,"[39.023291, -77.480859]",False
864899,Isabel Martínez,111091831332662628457,4.0,Entre semana el menú es genial: primer plato +...,"[Cafe, Pub]",106268661750829238054,1.356896e+09,Zombie Bar,None,"[40.423245, -3.70479]",False


In [21]:
minnX = 10e9
maxxX = -10e9
minnY = 10e9
maxxY = -10e9
cnt_outliers_lat  = 0 
cnt_outliers_lon  = 0 
t = 0 
for i in tqdm(range(len(temp_data))):
    if temp_data['gps'][i] is not None:
        t+=1
        if -180<= temp_data['gps'][i][0] <= 180:
            cnt_outliers_lat+=1
        if -180<= temp_data['gps'][i][1] <= 180:
            cnt_outliers_lon+=1
        minnX = min(minnX,temp_data['gps'][i][0])
        maxxX = max(maxxX,temp_data['gps'][i][0])
        minnY = min(minnY,temp_data['gps'][i][1])
        maxxY = max(maxxY,temp_data['gps'][i][1])

        
    
print(t,cnt_outliers_lat,cnt_outliers_lon)    
print(minnX,maxxX,minnY,maxxY)

100%|████████████████████████████████| 864901/864901 [00:17<00:00, 49698.50it/s]

862231 853755 853805
-53293013.0 68152045.0 -158187610.0 178017649.0


In [25]:
# Splitting latitude and longitute in 2 diffrent column

In [30]:
temp_data['lat'] = temp_data['gps'].apply(lambda x : x[0] if (x is not None) else -200)
temp_data['lon'] = temp_data['gps'].apply(lambda x : x[1] if (x is not None) else -200)

In [32]:
temp_data['gps']

0         [34.507753, -117.327177]
1          [40.819809, -74.000241]
2            [55.95843, -3.137685]
3          [44.665374, -65.610563]
4          [37.316856, 126.846288]
                    ...           
864896     [33.019414, -96.747739]
864897    [-41.263977, 173.920679]
864898     [39.023291, -77.480859]
864899       [40.423245, -3.70479]
864900      [30.454328, -86.63926]
Name: gps, Length: 864901, dtype: object

In [35]:
temp_data['lon']

0        -117.327177
1         -74.000241
2          -3.137685
3         -65.610563
4         126.846288
             ...    
864896    -96.747739
864897    173.920679
864898    -77.480859
864899     -3.704790
864900    -86.639260
Name: lon, Length: 864901, dtype: float64

In [36]:
## Dropping row if latitute and longitute not in range 

In [42]:
temp_data = temp_data.drop(temp_data[(temp_data.lat < -180 ) | (temp_data.lat > 180)].index)
temp_data = temp_data.drop(temp_data[(temp_data.lon < -180 ) | (temp_data.lon > 180)].index)

In [43]:
temp_data

,userName,gPlusUserId,rating,reviewText,categories,gPlusPlaceId,unixReviewTime,name,price,gps,closed,lat,lon
0,Justin Holland,104835831581429401434,4.0,Great job about a half hour wait for the $20.0...,[Car Wash],111799194080300135670,1.362161e+09,Civic Center Car Wash Inc,None,"[34.507753, -117.327177]",False,34.507753,-117.327177
1,ronald de leon,110331607471448368479,5.0,excellent sushi and great service.,"[Asian Restaurant, Japanese Restaurant, Produc...",116252055894950252701,1.359046e+09,Haru Sushi,$$$,"[40.819809, -74.000241]",True,40.819809,-74.000241
2,Michael Cairns,106185504400627678384,5.0,Have been using the team at Abercorn Plumbing ...,"[Plumber, Plumbing Supply Store]",105794257818107244229,1.349604e+09,Abercorn Plumbing & Heating,None,"[55.95843, -3.137685]",False,55.958430,-3.137685
3,Rose Brown,113712395831134054373,2.0,None,[Post Office],115302093695316020601,1.369516e+09,Clementsport Po,None,"[44.665374, -65.610563]",False,44.665374,-65.610563
4,전태현,108831174692729957795,5.0,괞찬음,[시외버스터미널],113786330221572304722,1.393477e+09,안산 종합여객자동차 터미널,None,"[37.316856, 126.846288]",False,37.316856,126.846288
...,...,...,...,...,...,...,...,...,...,...,...,...,...
864896,Bill Chen,107246420456791561435,5.0,None,"[Chinese Restaurant, Asian Restaurant]",113658954753439258377,1.375559e+09,Taiwan Cafe Inc,None,"[33.019414, -96.747739]",False,33.019414,-96.747739
864897,Frank Spychalski,105929453737619095652,5.0,"Super-friendly host, very nice house, perfect ...","[Budget Hotel, Hostel]",112448682523588624165,1.352096e+09,Anakiwa Backpackers & Green Caravan Cafe,None,"[-41.263977, 173.920679]",False,-41.263977,173.920679
864898,Phil Friesen,103571354677022020945,5.0,This store has an amazing selection of Asian f...,"[Asian Grocery Store, Supermarket]",118302020708442752531,1.374108e+09,Lotte Plaza - Ashburn,None,"[39.023291, -77.480859]",False,39.023291,-77.480859
864899,Isabel Martínez,111091831332662628457,4.0,Entre semana el menú es genial: primer plato +...,"[Cafe, Pub]",106268661750829238054,1.356896e+09,Zombie Bar,None,"[40.423245, -3.70479]",False,40.423245,-3.704790


In [46]:
filter_USA = temp_data.drop(temp_data[(temp_data.lat < 32.509762 ) | (temp_data.lat > 48.611122)].index)
filter_USA = filter_USA.drop(filter_USA[(filter_USA.lon < -120.080438 ) | (filter_USA.lon > -58.557279)].index)

In [44]:
#32.509762°   -120.080438°

In [45]:
#48.611122°-58.557279°

In [47]:
filter_USA

,userName,gPlusUserId,rating,reviewText,categories,gPlusPlaceId,unixReviewTime,name,price,gps,closed,lat,lon
0,Justin Holland,104835831581429401434,4.0,Great job about a half hour wait for the $20.0...,[Car Wash],111799194080300135670,1.362161e+09,Civic Center Car Wash Inc,None,"[34.507753, -117.327177]",False,34.507753,-117.327177
1,ronald de leon,110331607471448368479,5.0,excellent sushi and great service.,"[Asian Restaurant, Japanese Restaurant, Produc...",116252055894950252701,1.359046e+09,Haru Sushi,$$$,"[40.819809, -74.000241]",True,40.819809,-74.000241
3,Rose Brown,113712395831134054373,2.0,None,[Post Office],115302093695316020601,1.369516e+09,Clementsport Po,None,"[44.665374, -65.610563]",False,44.665374,-65.610563
5,Maria Benitez,108379517042616269418,5.0,None,"[Tack Shop, Entertainment]",104994189716832449897,1.352587e+09,Country Saddler LLC,None,"[35.176597, -79.467814]",False,35.176597,-79.467814
11,Mike Roberdeau,101605374547250760314,4.0,We had an excellent waiter who took very good ...,[American Restaurant],116107734350879152574,1.365949e+09,IHOP,$$$,"[38.986539, -94.464084]",False,38.986539,-94.464084
...,...,...,...,...,...,...,...,...,...,...,...,...,...
864885,dan king,101852593090718029957,5.0,One of the best auto shops I've been at recent...,"[Brake Shop, Car Inspection Station, Mechanic]",118293535998311708737,1.378501e+09,Right Choice Auto Service,None,"[43.887356, -79.437251]",False,43.887356,-79.437251
864887,Shelle Yolon,112059350052789258994,5.0,Teresa and Glenda (Team 6) did a fantastic job...,[House Cleaning Service],113618756892125017407,1.364321e+09,Maid Brigade of the South Bay of Los Angeles,None,"[33.789953, -118.337148]",True,33.789953,-118.337148
864890,Kentucky RiverRunners,102241121024674210076,5.0,We go here about twice a month. Both my wife a...,None,113638730559089261698,1.382992e+09,The Spaghetti Shop,None,"[38.353831, -85.791769]",False,38.353831,-85.791769
864896,Bill Chen,107246420456791561435,5.0,None,"[Chinese Restaurant, Asian Restaurant]",113658954753439258377,1.375559e+09,Taiwan Cafe Inc,None,"[33.019414, -96.747739]",False,33.019414,-96.747739


In [ ]:
## Now the real deal , filter 

In [48]:
merged['lat'] = merged['gps'].apply(lambda x : x[0] if (x is not None) else -200)
merged['lon'] = merged['gps'].apply(lambda x : x[1] if (x is not None) else -200)

In [49]:
merged = merged.drop(merged[(merged.lat < -180 ) | (merged.lat > 180)].index)
merged = merged.drop(merged[(merged.lon < -180 ) | (merged.lon > 180)].index)

In [50]:
filter_USA = merged.drop(merged[(merged.lat < 32.509762 ) | (merged.lat > 48.611122)].index)
filter_USA = filter_USA.drop(filter_USA[(filter_USA.lon < -120.080438 ) | (filter_USA.lon > -58.557279)].index)

In [51]:
filter_USA

,userName,gPlusUserId,rating,reviewText,categories,gPlusPlaceId,unixReviewTime,name,price,gps,closed,lat,lon
64,Jacquelyn Dorris,100000035085750632094,5.0,None,[College],104616184831707794985,1.331940e+09,Mt Sierra College,None,"[34.140884, -117.999102]",False,34.140884,-117.999102
65,David Mannington,104373107701194289732,2.0,I rather go to PCC. This school is money hungr...,[College],104616184831707794985,1.380309e+09,Mt Sierra College,None,"[34.140884, -117.999102]",False,34.140884,-117.999102
66,Victoria Negrete,105439490555882931029,3.0,None,[College],104616184831707794985,1.358380e+09,Mt Sierra College,None,"[34.140884, -117.999102]",False,34.140884,-117.999102
67,Mirna de Lira Lopez,107864646426078082690,5.0,If you are looking for a school that offers re...,[College],104616184831707794985,1.394746e+09,Mt Sierra College,None,"[34.140884, -117.999102]",False,34.140884,-117.999102
68,Robert Gutierrez,109158900084474679244,4.0,College is what you make of it. If you pay at...,[College],104616184831707794985,1.385525e+09,Mt Sierra College,None,"[34.140884, -117.999102]",False,34.140884,-117.999102
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8648980,Aj Boonstra,114675478704044213074,4.0,None,[State Park],110700880703640962323,1.352353e+09,Morrison-Rockwood State Park,None,"[41.848212, -89.962793]",False,41.848212,-89.962793
8648981,Aj Boonstra,114675478704044213074,5.0,Wouldn't think of going anywhere else. All but...,[Tattoo Shop],112810626989255988986,1.354917e+09,Sleeve Weasels Tatoo's,None,"[41.87253, -90.17982]",False,41.872530,-90.179820
8648989,Jey Atwill,114675539843034818123,5.0,I hired Scott to put replacement windows in my...,[Siding Contractor],112590524545945605174,1.362698e+09,SmithCo Exteriors,None,"[37.207388, -93.311498]",False,37.207388,-93.311498
8649001,Carly Rios,114675735342453210011,5.0,I have been with Serenity Insurance since May ...,[Auto Insurance Agency],107395757969573778019,1.355346e+09,Serenity Group Insurance,None,"[47.65458, -117.42627]",False,47.654580,-117.426270


In [53]:
filter_USA = filter_USA.reset_index(drop=True)

In [54]:
filter_USA.to_pickle(dataset_dir+ "USA_Filtered.pkl")  

In [55]:
filter_USA

,userName,gPlusUserId,rating,reviewText,categories,gPlusPlaceId,unixReviewTime,name,price,gps,closed,lat,lon
0,Jacquelyn Dorris,100000035085750632094,5.0,None,[College],104616184831707794985,1.331940e+09,Mt Sierra College,None,"[34.140884, -117.999102]",False,34.140884,-117.999102
1,David Mannington,104373107701194289732,2.0,I rather go to PCC. This school is money hungr...,[College],104616184831707794985,1.380309e+09,Mt Sierra College,None,"[34.140884, -117.999102]",False,34.140884,-117.999102
2,Victoria Negrete,105439490555882931029,3.0,None,[College],104616184831707794985,1.358380e+09,Mt Sierra College,None,"[34.140884, -117.999102]",False,34.140884,-117.999102
3,Mirna de Lira Lopez,107864646426078082690,5.0,If you are looking for a school that offers re...,[College],104616184831707794985,1.394746e+09,Mt Sierra College,None,"[34.140884, -117.999102]",False,34.140884,-117.999102
4,Robert Gutierrez,109158900084474679244,4.0,College is what you make of it. If you pay at...,[College],104616184831707794985,1.385525e+09,Mt Sierra College,None,"[34.140884, -117.999102]",False,34.140884,-117.999102
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3078377,Aj Boonstra,114675478704044213074,4.0,None,[State Park],110700880703640962323,1.352353e+09,Morrison-Rockwood State Park,None,"[41.848212, -89.962793]",False,41.848212,-89.962793
3078378,Aj Boonstra,114675478704044213074,5.0,Wouldn't think of going anywhere else. All but...,[Tattoo Shop],112810626989255988986,1.354917e+09,Sleeve Weasels Tatoo's,None,"[41.87253, -90.17982]",False,41.872530,-90.179820
3078379,Jey Atwill,114675539843034818123,5.0,I hired Scott to put replacement windows in my...,[Siding Contractor],112590524545945605174,1.362698e+09,SmithCo Exteriors,None,"[37.207388, -93.311498]",False,37.207388,-93.311498
3078380,Carly Rios,114675735342453210011,5.0,I have been with Serenity Insurance since May ...,[Auto Insurance Agency],107395757969573778019,1.355346e+09,Serenity Group Insurance,None,"[47.65458, -117.42627]",False,47.654580,-117.426270
